In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import time
from pprint import pprint
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import json

pd.set_option('max_colwidth', 1000)

In [2]:
options          = webdriver.ChromeOptions()
options.headless = True


In [ ]:
car_urls = pd.read_pickle("cars_carwale6.pkl")
print(car_urls.shape)
car_urls.head()

In [4]:
car_details = []

In [ ]:
total = len(car_urls)
print(f"{total=}")

In [ ]:
for i in range(401,len(car_urls)): 
    url = car_urls['carURLs'][i]
    # print(requests.head(url).status_code)
    # if requests.head(url).status_code != 200: 
    #     continue
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    time.sleep(5)

    soup    = BeautifulSoup(driver.page_source, 'html.parser')
    scripts = soup.find_all('script')
    driver.quit()

    script_text = [script.text for script in scripts if 'window.__INITIAL_STATE__' in script.text][0]
    pattern     = '{.*};'
    json_text   = re.findall(pattern, script_text)[0][:-1]

    #print(json_text)
    json_data = json.loads(json_text)
    if json_data.get('usedCarDetails', False):
        make              = json_data['usedCarDetails']['carInfo']['data'].get('makeName','MISSING')
        model             = json_data['usedCarDetails']['carInfo']['data'].get('modelName','MISSING')
        variant           = json_data['usedCarDetails']['carInfo']['data'].get('versionName','MISSING')
        year              = json_data['usedCarDetails']['carInfo']['data'].get('makeYear','MISSING')
        transmission      = json_data['usedCarDetails']['carInfo']['data'].get('transmissionType','MISSING')
        bodyType          = json.loads(json_data['metaKeyWords']['pageSchema']).get('bodyType','MISSING')
        fuelType          = json_data['usedCarDetails']['carInfo']['data'].get('fuelName','MISSING')
        ownerNumber       = json_data['usedCarDetails']['carInfo']['data'].get('noOfOwners','MISSING')
        #odometerReading   = json.loads(json_data['metaKeyWords']['pageSchema']).get('mileageFromOdometer')[0].get('value','MISSING')
        odometerReading   = json_data['usedCarDetails']['carInfo']['data'].get('kilometers','MISSING')
        cityRto           = json_data['usedCarDetails']['carInfo']['data'].get('registrationNumber','MISSING')
        listingPrice      = json_data['usedCarDetails']['carInfo']['data'].get('priceNumeric','MISSING')
        fitnessUpto       = json_data['usedCarDetails']['carInfo']['data'].get('fitnessUpto','MISSING')
        insuranceType     = json_data['usedCarDetails']['carInfo']['data'].get('insurance','MISSING')
        duplicateKey      = json_data['usedCarDetails']['carInfo']['data'].get('duplicateKey','MISSING')
        city              = json_data['usedCarDetails']['carInfo']['data'].get('cityName','MISSING')
        registrationYear  = json_data['usedCarDetails']['carInfo']['data'].get('registrationDate','MISSING')
        registrationMonth = json_data['usedCarDetails']['carInfo']['data'].get('registrationMonth','MISSING')
        websiteUrl        = url

        # print(f"{make=}")
        # print(f"{model=}")
        # print(f"{variant=}")
        # print(f"{year=}")
        # print(f"{transmission=}")
        # print(f"{bodyType=}")
        # print(f"{fuelType=}")
        # print(f"{ownerNumber=}")
        # print(f"{odometerReading=}")
        # print(f"{cityRto=}")
        # print(f"{listingPrice=}")
        # print(f"{fitnessUpto=}")
        # print(f"{insuranceType=}")
        # print(f"{duplicateKey=}")
        # print(f"{city=}")
        # print(f"{registrationYear=}")
        # print(f"{registrationMonth=}")
        # print(f"{websiteUrl=}")

        car_details.append([make,model,variant,year,transmission,bodyType,fuelType,ownerNumber,odometerReading,cityRto,listingPrice,fitnessUpto,insuranceType,duplicateKey,city,registrationYear,registrationMonth,websiteUrl])
        print(f"waiting 5 seconds before next car scan---------------------------------------------------{i:04}/{total}")
        time.sleep(1)
        if i > 0 and (i%100 == 0 or i == len(car_urls)-1):
            car_details_cars24 = pd.DataFrame(car_details, columns=['make','model','variant','year','transmission','bodyType','fuelType','ownerNumber','odometerReading','cityRto','listingPrice','fitnessUpto','insuranceType','duplicateKey','city','registrationYear','registrationMonth','websiteUrl'])
            car_details_cars24.to_pickle(f"car_details_carwale/06/car_details_carwale_{i}.pkl")
            print(f"car_details_carwale_{i}.pkl saved successfully")
            car_details = []
    else: continue

In [ ]:
car_details_cars24 = pd.DataFrame(car_details, columns=['make','model','variant','year','transmission','bodyType','fuelType','ownerNumber','odometerReading','cityRto','listingPrice','fitnessUpto','insuranceType','duplicateKey','city','registrationYear','registrationMonth','websiteUrl'])
car_details_cars24.to_pickle(f"car_details_carwale/06/car_details_carwale_{i}.pkl")
print(f"car_details_carwale_{i}.pkl saved successfully")
car_details = []